# Metadata Module

**This is a literate notebook.**

## Motivation

From [Wikipedia](https://en.wikipedia.org/wiki/Metadata)
> Metadata is "data that provides information about other data". In other words, it is "data about data."

What were the conditions on a particular day?  The crew?  What sort of jib settings did we use?  The finishing position?  What were the shroud settings?  How did they perform?  

I started out by writing a email for each race, trying to including learnings, conditions, results.  I moved to creating a Google doc for each race, easier to edit and update. And then I moved to creating a Jupyter notebook for each race day, easier to include data from the actual race all in one place.

The problems with these approaches:

- Repeated work.  Each email/gdoc/notebook is a vague copy of the previous, updated with new info.  This copy/edit process is annoying.
  - For example, one step is to grab the weather/tides, and just this step takes a while by hand.
- The data is locked in a human readable document, not in a machine readable representation.
  - No easy way to generate a single document (i.e. table of contents that shows all races, dates and times).
- No way to analyze the data in one place.  Where can we look to see trends or issues that are inconsistent?

Philosophically, I like metadata which can be searched and cross-referenced.  Data should be easy to edit and update and view.

The solution is to store all this metadata in a single easy to edit datastructure which can then be analyzed/created/edited/rendered for various needs.

## Overview

Code to process race metadata and associate with race logs.

There are currently 3 sources of hand entered metadata, hopefully fewer soon:
- A file called metadata.yaml (in YAML).  This is the final ultimate source for metadata.
  - YAML is super powerful, allowing you to enter complex structured info.
  - It is also designed to be human readable (unlike a database file, or even a CSV).
  - Unfortunately there is no strong schema, and it is not that hard to mess things up.
- A google spreadsheet (gsheet) that is fed by a [Google Form](https://forms.gle/JENZZdSWKNuoF8icA) (which is easy to use on the ride home from the race).
  - The form determines the schema, which can be changed, but it does enforce some structure.
  - The spreadsheet can be downloaded as a CSV from a URL.
- An older pandas dataframe, `log_info.pd` which is now deprecated.

And a final source, which is a default and empty metadata record generated when we first upload the file.

In all cases above the **key** is the date (and we therefore assume that there is a single "file" per day).  In practice we may have several races on a single day, though these will be in one file. The YAML file will support the ability to discuss the segments.  The Google form does not.

**How to merge duplicates?**. 

- Multiple rows with same date in the gsheet.
  - Delete by hand?  Take the latest?
- Same date in gsheet and YAML.
  - Note, gsheet row will move to YAML when it first appears.
  - Figure out which is newer.  
    - If YAML is newer, keep it.
    - If gsheet is newer, then keep **both**.  Warn user and ask to edit.

  
  

### References 

- Good YAML reference to start: [YAML tutorial](https://rollout.io/blog/yaml-tutorial-everything-you-need-get-started/)
  - [The official reference](https://yaml.org/) Its written in YAML (which makes it a bit weird).
- Nice Google page on [how to use Google Forms](https://zapier.com/learn/google-sheets/how-to-use-google-forms/)


## TODO

- Add timestamp to all YAML entries?  

- Tides? 

- Currents?  

- Weather: wind, etc.  Weather buoy?

- Create a page before the race? 

- Phone images captured during the race 
  - pull them in, link them to the map?
  - extract settings?
  
- Sometimes there are two files from the same day.  In general should not have happened...  but it screws things up.

- Provide a tool to slice a days data into "segments".



## Caveats and concerns

- YAML, as edited by a human author, does not support a strong schema.  Its easy to mess things up, with typos, missing fields, incorrectly named fields, etc.


In [1]:
# imports
import os
import copy
import numbers
import re

import yaml  # We'll use YAML for our metadata
import json

import arrow
import numpy as np
import pandas as pd

# These are libraries written for RegattaAnalysis
from global_variables import G  # global variables
import utils
from utils import DictClass
import process as p
from nbutils import display_markdown, display

In [2]:
# notebook - YAML race metadata example.

# Below is snippet of YAML inline.  I am not going to document YAML here.  But notice that the structure
# is reminiscent of Python itself, and it is somewhat readable.

example = """
file: 2020-04-16_14:54.pd.gz
date: "2020-04-16"
title: Tune-up with Creative
purpose: tune_up
conditions: >-
  Beautiful day. Winds were 3 quickly building to 10ish. Flat
  seas. Upwind to Pt. Wells buoy, raised and raced home to the hamburger.
performance: >-
  Good height and speed vs. Creative on the way upwind. Perhaps a bit
  slow at first downwind, exploring to tradeoffs between depth and
  speed.  Best downwind speed when I was at the shrouds and Sara had a
  hand on the mainsheet.
learnings: >-
  Let the sails out for downwind: both main and kite.  Stand forward
  if possible.

  Shroud settings seemed really great, and versatile.  With only 2 on
  the boat, we sailed very well.  These settings are the new base!
raceqs_video: "https://youtu.be/9a5bLeZw8EM"
raceqs: "https://raceqs.com/tv-beta/tv.htm#userId=1146016&divisionId=64190&updatedAt=2020-04-17T18:05:59Z&dt=2020-04-16T15:43:47-07:00..2020-04-16T17:39:12-07:00&boat=Creative"
segments:
  - winds: [6, 12]
    tensions: [29, 10, 0]
    port: [2.251, 1.953, 999]
    stbd: [2.271, 1.959, 999]
    thoughts: >-
      Overall we have had trouble with the Quantum quick tune card,
      where the uppers are a bit too loose or the middles too tight.
      The result is that the mast falls off at the top, rather than
      staying straight or sagging for power.  We took a bit off the
      middle (12 down to 10).
questions:
  - text: Was the prop set correctly?
    author: sara
    context: Were we slower on one tack than the other? 
    proposed_solution: ??
"""

# The data can be trivialy read in this way.  A simple Python datastructure results, in this case
# a dictionary.  
race_metadata = yaml.load(example, Loader=yaml.Loader)

display_markdown("Notice that `segments` and `questions` are sublist of dicts.")
display(race_metadata)

Notice that `segments` and `questions` are sublist of dicts.

{'file': '2020-04-16_14:54.pd.gz',
 'date': '2020-04-16',
 'title': 'Tune-up with Creative',
 'purpose': 'tune_up',
 'conditions': 'Beautiful day. Winds were 3 quickly building to 10ish. Flat seas. Upwind to Pt. Wells buoy, raised and raced home to the hamburger.',
 'performance': 'Good height and speed vs. Creative on the way upwind. Perhaps a bit slow at first downwind, exploring to tradeoffs between depth and speed.  Best downwind speed when I was at the shrouds and Sara had a hand on the mainsheet.',
 'learnings': 'Let the sails out for downwind: both main and kite.  Stand forward if possible.\nShroud settings seemed really great, and versatile.  With only 2 on the boat, we sailed very well.  These settings are the new base!',
 'raceqs_video': 'https://youtu.be/9a5bLeZw8EM',
 'raceqs': 'https://raceqs.com/tv-beta/tv.htm#userId=1146016&divisionId=64190&updatedAt=2020-04-17T18:05:59Z&dt=2020-04-16T15:43:47-07:00..2020-04-16T17:39:12-07:00&boat=Creative',
 'segments': [{'winds': [6, 1

### Render this data to make it more readable

We can prettily easily write code that "renders" these Python structures into readable Markdown.

In [3]:
# code to render a metadata entry in markdown... 

def display_race_metadata(race_record, include_extras=True):
    "Summarize a race."
    display_markdown(f"# {race_record['title']}: {race_record['date']}")
    rr = race_record.copy()
    for k in "description conditions performance learnings".split():
        if k in rr:
            display_section(k.capitalize(), rr.pop(k))
    links = "raceqs raceqs_video".split()
    if has_key(links, race_record):
        display_markdown("## Links")
        lines = ""
        for k in links:
            if k in rr:
                lines += lines_url(make_title(k), rr.pop(k))
        display_markdown(lines)
    if include_extras:
        keys = list(rr.keys())
        if len(keys) > 0:
            lines = ""
            display_markdown("## Extras")
            lines = lines_dict("", keys, rr)
            display_markdown(lines)

def has_key(key_list, dictionary):
    for k in key_list:
        if k in dictionary:
            return True
    return False

def is_list_of_dicts(val):
    return isinstance(val, list) and isinstance(val[0], dict)

def lines_dict(prefix, keys, dictionary):
    lines = ""
    for k in keys:
        val = dictionary[k]
        if is_list_of_dicts(val):
            for i, v in enumerate(val):
                lines += f"{prefix}- **{k}: {i}**\n"
                lines += lines_dict(prefix+"  ", v.keys(), v)
        else:
            lines += f"{prefix}- **{k}**: {val}\n"
    return lines
            
def display_section(title, text):
    "Displays a markdown section with text."
    display_markdown(f"## {title}")
    display_markdown(text)        
        
def is_url(s):
    return s.startswith("http")  # Not great, but OK for now

def make_title(key):
    """"
    YAML keys are python keywords (lowercase and separated by underscores).  This converts to a pretty 
    and printable string.
    """
    words = key.split("_")
    words = [w.capitalize() for w in words]
    return " ".join(words)

def lines_url(link_text, url):
    "Displays a markdown URL."
    return f"- [{link_text}]({url})\n"

In [4]:
# notebook - Create a human readable version of this metadata
display_race_metadata(race_metadata)

# Tune-up with Creative: 2020-04-16

## Conditions

Beautiful day. Winds were 3 quickly building to 10ish. Flat seas. Upwind to Pt. Wells buoy, raised and raced home to the hamburger.

## Performance

Good height and speed vs. Creative on the way upwind. Perhaps a bit slow at first downwind, exploring to tradeoffs between depth and speed.  Best downwind speed when I was at the shrouds and Sara had a hand on the mainsheet.

...

Let the sails out for downwind: both main and kite.  Stand forward if possible.
Shroud settings seemed really great, and versatile.  With only 2 on the boat, we sailed very well.  These settings are the new base!

## Links

- [Raceqs](https://raceqs.com/tv-beta/tv.htm#userId=1146016&divisionId=64190&updatedAt=2020-04-17T18:05:59Z&dt=2020-04-16T15:43:47-07:00..2020-04-16T17:39:12-07:00&boat=Creative)
- [Raceqs Video](https://youtu.be/9a5bLeZw8EM)


## Extras

- **file**: 2020-04-16_14:54.pd.gz
- **date**: 2020-04-16
- **title**: Tune-up with Creative
- **purpose**: tune_up
- **segments: 0**
  - **winds**: [6, 12]
  - **tensions**: [29, 10, 0]
  - **port**: [2.251, 1.953, 999]
  - **stbd**: [2.271, 1.959, 999]
  - **thoughts**: Overall we have had trouble with the Quantum quick tune card, where the uppers are a bit too loose or the middles too tight. The result is that the mast falls off at the top, rather than staying straight or sagging for power.  We took a bit off the middle (12 down to 10).
- **questions: 0**
  - **text**: Was the prop set correctly?
  - **author**: sara
  - **context**: Were we slower on one tack than the other?
  - **proposed_solution**: ??


In [5]:
# Race metadata is stored as a multi-document sequence in the YAML file.  

def read_metadata():
    """
    Read the race metadata and return a struct, with a 
    - timestamp
    - records:  list of records
    - dates:    dict from date to record
    """
    race_yaml = read_yaml(G.METADATA_PATH)
    dates = {}
    records = []
    for record in race_yaml:
        # If the record is missing a source, assume it was written byhand.
        if 'source' not in record:
            record['source'] = 'byhand'
        if 'date' not in record:
            print(record)
        dates[record['date']] = record
        records.append(record)
    # File timestamp, used to find valid updates in other sources.
    ts = arrow.get(os.path.getmtime(G.METADATA_PATH)).to('US/Pacific')
    G.logger.info(f"Read {len(records)} records.")
    return DictClass(dates=dates, records=records, timestamp=ts)

def save_metadata(race_records):
    """
    Save a sequence of race records to the metadata file.
    """
    G.logger.info(f"Writing {len(race_records)} records.")    
    utils.backup_file(G.METADATA_PATH)
    # For convenience sort the records by date before writing.
    sorted_records = sorted(race_records, key=lambda r: r['date'])
    with open(G.METADATA_PATH, 'w') as yfs:
        save_yaml(sorted_records, yfs)

def save_json(race_records, json_file):
    """
    Save a sequence of race records to JSON.
    """
    G.logger.info(f"Writing {len(race_records)} records.")    
    utils.backup_file(G.METADATA_PATH)
    # For convenience sort the records by date before writing.
    sorted_records = sorted(race_records, key=lambda r: r['date'])
    with open(json_file, 'w') as fs:
        json.dump(sorted_records, fs) 

# A bit of magic here to ensure we have the best loader/dumper.  Specifying this is required when 
# calling load/dump (below).
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

def read_yaml(yaml_path):
    "Read the race records stored in the YAML file.  Return a dict indexed by date string: YYYY-MM-DD."
    with open(yaml_path, 'r') as yaml_stream:
        race_yaml = list(yaml.load_all(yaml_stream, Loader=Loader))
    return race_yaml

                     
def save_yaml(race_entries, stream=None):
    "Save the race metadata as YAML."
    return yaml.dump_all(race_entries, stream, Dumper=Dumper,
                         default_flow_style=False, sort_keys=False)

In [6]:
# notebook - load some metadata

metadata = read_metadata()
display(metadata.timestamp)

display_race_metadata(metadata.dates['2020-04-19'])
display_race_metadata(metadata.dates['2020-06-06'])


2022-04-21 07:54:19,993|INFO|read_metadata| Read 179 records.


<Arrow [2022-04-17T18:31:58.798000-07:00]>

# Tune-up with Creative: 2020-04-19

## Description

## Conditions

...

Slower in the up and down but we took a different path

## Learnings

Bilge pump button was in middle again. Maybe our current prediction distracting us?

## Extras

- **file**: 2020-06-06_09:23.pd.gz
- **date**: 2020-06-06
- **title**: Cohab #5
- **begin**: 36271
- **end**: 152106
- **source**: byhand
- **purpose**: race
- **crew**: ['Sara', 'Paul']
- **wave**: 1-2
- **wind**: [8, 14]
- **port_pointing**: 25-27
- **stbd_pointing**: 25-27
- **settings**: Mostly loose.  Left back stay on too long down wind
- **shroud_name**: Light breeze
- **shroud_tension**: [27, 12, 0]
- **timestamp**: 2020-06-06 22:13:15
- **fluids**: Empty Bilge?
- **shroud_tensions**: 27,12,0


In [7]:
# Since the data for all days is in one place its easy to generate an all up summary.

def race_summary(race):
    display_markdown(race_summary_lines(race))

def race_summary_lines(race):
    lines = ""
    lines += f"- **{race['date']}**: {race['title']}\n"
    for key in "description conditions".split():
        if key in race and race[key] is not None and len(race[key]) > 0:
            lines += f"  - *{key.capitalize()}:* {race[key]}\n"
    return lines

def display_race_summaries(race_records):
    "Summarize each race."
    lines = ""
    for race in race_records:
        lines += race_summary_lines(race)
    display_markdown(lines)

def summary_table(race_records, columns = None):
    "Return a summary table the races."
    rows = []
    if columns is None:
        columns = "date title file source".split()
    for race in race_records:
        row = {k:race.get(k, '') for k in columns}
        rows.append(row)
    return pd.DataFrame(rows)


In [8]:
# notebook 

metadata = read_metadata()

display_race_summaries(metadata.records)
summary_table(metadata.records)

2022-04-21 07:54:23,027|INFO|read_metadata| Read 179 records.


- **2019-10-04**: Sail to Everett for Foulweather Bluff Race.
- **2019-10-05**: Foulweather Bluff Race
  - *Description:* Foulweather Bluff Race
- **2019-10-11**: Short practice, upwind tacks and downwind jibes.
- **2019-10-12**: CYC PSSC Day 1
  - *Description:* CYC PSSC Day 1
- **2019-10-18**: Short, at dock.
- **2019-10-19**: STYC Fall Regatta
  - *Description:* STYC Fall Regatta.
- **2019-10-26**: Grand Prix Saturday
  - *Description:* Grand Prix Saturday.
- **2019-11-07**: Short, at dock.
- **2019-11-16**: Snowbird #1
  - *Description:* Snowbird #1.
- **2019-11-23**: Practice.
- **2019-11-24**: Practice.
- **2019-11-29**: Longer.  At dock for data collection.
- **2019-12-06**: Short, at dock.
- **2019-12-07**: Snowbird #2
  - *Description:* Snowbird #2.
- **2020-01-18**: Practice
- **2020-01-25**: STYC Iceberg
  - *Description:* Lot's of wind was predicted, but we got little.  We swapped on older sails, and the rig was super tight.  WE WERE SLOW!
- **2020-02-08**: Snowbird #4
  - *Description:* Snowbird #4.
- **2020-02-16**: 
- **2020-02-29**: Practice
- **2020-03-07**: CYC, Blakely Rocks, CSS
  - *Description:* Great day!  Good speed, both upwind and down.
- **2020-03-14**: 
- **2020-03-21**: 
- **2020-03-22**: 
- **2020-03-31**: 
- **2020-04-04**: 
- **2020-04-08**: 
- **2020-04-12**: Tune Up
  - *Description:* Quick tune up race vs. Creative.  We raced upwind from West Point to Meadow Point.  They were definitely a bit faster.
- **2020-04-16**: Tune-up with Creative
  - *Conditions:* Beautiful day. Winds were 3 quickly building to 10ish. Flat seas. Upwind to Pt. Wells buoy, raised and raced home to the hamburger.
- **2020-04-19**: Tune-up with Creative
  - *Conditions:* Beautiful day. Very light at times, almost glassy.
- **2020-04-24**: Tune-up with Creative
  - *Description:* Sailed up from Elliott Bay. Met Creative in the middle.
  - *Conditions:* Super light winds from the SSE (1-3) during tuneup.  Gave up a bit early.  After, on the way back, winds picked up to 8-10.
There should have been a strong current heading south, due to a large flood.  The interactions between southerly current and winds from the south may have been complex.
- **2020-04-26**: Return from Elliott Bay
- **2020-04-28**: Fun sail.
  - *Conditions:* Overcast.  Calm as we arrived on dock, but then it picked up to 10-15.  A bit rough.
- **2020-05-01**: 
- **2020-05-04**: Raises and Douses, part 2.
  - *Conditions:* Light winds and then building.
- **2020-05-06**: Out with Marisa
  - *Description:* Went out with Marisa to practice for cohabitation race
  - *Conditions:* 16 kits consistent.  Some sea state.
- **2020-05-08**: Practice
  - *Description:* Good day at about 13 knots.   Nice weather.  Just before high tide.  Current pushing out as we left the dock. Just Sara Paul on the boat.  We went pretty far up north doing tuneup. We needed Cunningham and back stay on. We have the rig tune for base setting  10 to 40 nots.  We tried the leward spin drop for the first time.  10 knots as we dropped. 
  - *Conditions:* Weird wind shift at the end of our excursion 
- **2020-05-09**: Cohabitate Race
  - *Description:* Great day on the water. Range from 10-15 Jordi of the day. Marisa came along with us.  We had a pretty major disappointment when we tried to drop the spinnaker using the letter box.
  - *Conditions:* Oscillating wind from the north.  
- **2020-05-11**: Practice 
  - *Description:* Goal was to try letterbox again but conditions were challenging. 
  - *Conditions:* Overcast with swirling wind from all four directions.  Low tide at -1.9, with wind from the south mostly.  
- **2020-05-15**: Practice with Beth and Holm
  - *Description:* Friday evening sunset sail.  Did some raises and douses
  - *Conditions:* Nice. Warm
- **2020-05-16**: Cohab 2 race
  - *Description:* Paul organized this race. The course was nwmn That means hamburger, West Point buoy , Meadowpoint, hamburger. It was supposed to be much longer but there was not much wind
  - *Conditions:* Very light to no wind at times. Strong current to start pushing us 1.2 knots south. Wind came from south and east and north.  It was hard to figure out when to use the spinnaker
- **2020-05-18**: 
- **2020-05-23**: Cohab 3
  - *Description:* Nwmn and we went to West Point again.  We tried hard to use current data but it was hard. Extra hard rounding at meadow poi t due to southern current. 
  - *Conditions:* Light wind from the south. 
- **2020-05-27**: 2020-05-27
- **2020-05-29**: Practice 
  - *Description:* We did  nmnmn.  Practiced windward douse twice and one Mexican.  Wind from the nor  th and current pushing south
  - *Conditions:* Nice!  Wind went up and down rapidly. 
- **2020-05-31**: Cohab #4
  - *Description:* Solid day of racing against “More Uff Da” and “Jaded”.  Victoria was out there too. Raining at points but dried up on the long leg. Course was nbwmnun. Our start was 10:26:19am. Finished at 1:12.  I think we won!  But still need to calculate the handicap. 
  - *Conditions:* Breeze started from the south as we warmed up but clocked around to the East so we raised spin at start on a port tack to Ballard buoy and stayed on spin to West Point.  Mexican at West Point and  went north on stbd. 
- **2020-06-01**: Monday race
  - *Description:* Creative and Jubilee and us made for a good race
- **2020-06-05**: Friday Eve practice 
  - *Description:* Light wind mild day.  Didn’t get very far but worked on job car settings 
  - *Conditions:* Warm. Light wind.  High tide
- **2020-06-06**: Cohab #5
  - *Description:* Course nmwnubn.  Our star was 10:29:26. 
  - *Conditions:* Oscillating wind with two storm clouds passing over Bainbrige. Puffs and lulls.  Wind varied from 160 to 230.  Numbers in 200s were coincident with storms.  We tried to use the drllenbaugh approach calling out persistent vs oscillating times.   We still got beat. Not sure where but near meadow point on way back was more clear 
- **2020-06-08**: TYT Monday 
  - *Description:* Rough shifting wind from south with current churn.  Jubalie, Liftoff’, Creative, and us among the J105s. Many other boats too.  We. Beat jubilee and creative but not left off. They were a couple mins ahread. Eclipse was in the mix too.  Gusto got a new phrf and maybe not flying.  Our start was 6;35. Finish 7:55
  - *Conditions:* Rolling and rough ride!  Rather fun in retrospect but a bit scary at times. 
- **2020-06-10**: Sunset Sail
  - *Description:* Sailed to Elliot Bay to look For a good race route.   Wind picked up on way back so we floated out spin at sunset.  Just beautiful!
  - *Conditions:* Gentle south west wind
- **2020-06-13**: Cohab#6 
  - *Description:* Crazy wind shifty day.  Started south to meadow point on spin. West Point.  Hamburger. At most to meadow point again on spin when wind shifted north and we had to drop. Almost hoisted spin again 10 mins later just to decide to wait again.  We got out ahead of More Jubalee and Lift Off on second leg and they never caught us. Only Valkyrie caught us but not by much!  Then big storm hit and we called it early. Fun day!
  - *Conditions:* Wind oscillating 160-230. Then flipped 330-012
- **2020-06-15**: TYT Monday 
  - *Description:* Windy and rougher conditions than usual.  We lost to Creative mostly because our rounding were imperfect.  They also went lower and at least as fast down wind.  
  - *Conditions:* Lots of puffs.  We would see 11 knots on gauge but only going 4.5.  So I’d have to foot off to get speed but then the heel was so big the rail was buried. Downwind was fun!  
- **2020-06-19**: Friday practice 
  - *Description:* Nice evening sail.  80 degrees.  No competition to race so we just did laps and some experiments with drops
  - *Conditions:* Breezy 
- **2020-06-20**: Cohab 7. Downtown Exploration
  - *Description:* Fresh breeze day for the race we created.  NW—spaceneedlebellharborline-NW. 
  - *Conditions:* Nice wind 10-12 up wind. 150-174 from south.    Picked up to 15-20  for the exciting spinnaker finish with 4 boats side by side.  We finished behind Lodos and needed a referee on double trouble and eclipse.  
- **2020-06-22**: TYT Monday
  - *Description:* Beautiful summer evening.  Course was NUN. Our start was 6:40.  We either had a great start or over early.  But our upwind speed was not excellent.  Biggish waves meant we had to foot off particularly on starboard.  Moose, Jubilee, Creative, Liftoff, and Gusto came out too.  
  - *Conditions:* Breezy with some sea state.  
- **2020-06-24**: Gas run
  - *Description:* Mad dash to the fuel dock to get deisel since they are now open to 5:00 
- **2020-06-27**: Cohab 8
  - *Description:* NBWNUN.  Folks worked super hard to “knock the dust off” and do very well.  Might have actually helped to be shorthanded, since we needed 100% attention 100% of the time.   We focused intensely on Moose, a luxury since there were no other boats nearby (Rush was ahead the whole time!).  We marked them and beat them to U mark, and then separated 20+ boat lengths.  Congrats.   Tacks were great. Jibes were flawless.  Raise was great.  Douse worked well.  Folks did their jobs, with little practice and little review.  And no critical job got dropped.  Props to Amy for working the PIT, which has dozens of little steps (in addition to main… and we were pretty fast).   Thanks to you both for showing up early and for keeping cheerful through the near constant rain.  Almost July 4th and it was darn cold out there!  (Double thanks for Steve for getting there first and getting the boat almost completely set!)  
  - *Conditions:* Weird East wind predicted.  Did not disappoint at the start.  Converged to a northerly at 6-10.  Very light at first, but then consistent.  Consistent rain and low visibility.  Hard to find the marks.
- **2020-06-29**: TYT Monday night 
  - *Description:* Fresh breeze out there but minimal waves. Quite a nice day for sailing. 
  - *Conditions:* 10-18 knots. Liftoff, corvo, creative, moose and us out there among the 105s.  We had a good start on starboard and tacked soon to port. 
- **2020-07-06**: 2020-07-06
- **2020-07-08**: CYC Wednesday #1
  - *Description:* Double handed race.  Course JMEFMEF. Pronounced J-mef-mef. That’s just our practice workout sail using the green can and meadow point.   
- **2020-07-11**: Cohab #10
  - *Description:* Course: NWNUN. Wind from 0-15 knots. Tide ebbing throughout from 8.5ft. Nice calm sea state.  10 boats out.  4 j105s. Jubilee,, Moose, and Jaded came with.  LA and Dan double handed Jaded and Erik and Patty double handed Jubi
  - *Conditions:* Wind from south. 140-200 oscillating. Some other boats out so the field is getting more crowded. 
- **2020-07-13**: STYC Monday 
  - *Description:* Beautiful evening sail.  5 of the 105s came out. Corvo, Jubilee, creative, liftoff and us.  Marisa made lovely green masks for us.  
  - *Conditions:* Wind from true north.  Ranged from 5-13 knots.  Some one knot current push sometimes
- **2020-07-15**: CYC Wednesday 2
  - *Description:* We had Jaded and Corvo out with us.   All double handed. Started north of meadow point, north to Spring Beach, south to green can and back to start. Fresh breeze 16-18 at times. Note the big lift going north up the coast.  Wind was straight north at meadow but more like 338 near mark. 
  - *Conditions:* Steady wind. Minimal current change.  
- **2020-07-18**: Cohab #10 for real
  - *Description:* Course nmwnmw
  - *Conditions:* Wind was very light to begin.  Picked up to 11 eventually at end of race. Took 3 hours to do our two laps.  Strong ebbing tide. Jeff Madrigali was on board with us double handing. 
- **2020-07-20**: Ballard Cup #3
  - *Description:* Great day on water.  Sunny and warm. Windy was 6-12 mostly.  One painful lull on last leg. 
- **2020-07-22**: CYC Wednesday 
  - *Description:* Lighter wind than expected
  - *Conditions:* Windy shifted just before the start. 
- **2020-07-25**: Women at the Helm
  - *Description:* 3 races. NMBMN,  NMEN,  NRN.   We flat out won the second race. Really good speed. First and third race we got fouled and didn’t call it right. Poke hit our stbd aft corner bow to stern pulpit as we rounded R.  We had mark room rights.  It looked like minor damage so I kept sailing.  They did turns.   Reckless on the other van fouled us as the port boat while we were stbd. I ducked them and yelled.  They said first time driver and then proceeded to win our division.  I should have insisted on turns.  Will you take a penelty turn ?  I will call protest.  Pressure was better outside.  Most of the day.  Third race was exBrought out the pink spin and new jib. 
  - *Conditions:* Beautiful sunny day.  Ebbing tide all day.  Wind was light in the morning so our shrouds were a bit tight.  Spot on for race 2.  Wind picked up later so we went 7 both up and downwind. Speed was our friend.  
- **2020-07-27**: Ballard cup Monday 
  - *Description:* Nice evening.  A lot of boats out again.  Course NMWMN. Flooding current.  Got hit hard after West Point.   Over 1knot against us. We tried both in and out. The Meadow Point hurricane worked in our favor and lifted us up the beach nicely so made the mark when I thought we could not.  Over shot a bit in fact. 
  - *Conditions:* Sunny warm moderate breeze
- **2020-07-29**: CYC Wednesday 
  - *Description:* Short sail JMWJ. 
- **2020-07-31**: Picnic Sail Friday
  - *Description:* Moderate Breeze and tide ebbing against the wind so rather bumpy upwind. Nice easy sailing. 
- **2020-08-01**: Picnic Sail Friday
  - *Description:* Moderate Breeze and tide ebbing against the wind so rather bumpy upwind. Nice easy sailing. 
- **2020-08-03**: 2020-08-03
- **2020-08-08**: Double practice day!!
  - *Description:* Went out Saturday morning with light air and full crew for practice. Worked on 3 boat length circle with rope and roll tacks.  Then Paul and I went out again so he could try single handing. 
- **2020-08-08**: Double practice day!!
  - *Description:* Went out Saturday morning with light air and full crew for practice. Worked on 3 boat length circle with rope and roll tacks.  Then Paul and I went out again so he could try single handing. 
- **2020-08-10**: Ballard Cup Monday
  - *Description:* Wind forecast was 10 to 19 nots. So we said the shrouds at bass. But it might’ve been a little bit too tight we were slow on the upwind. Nice weather. The course was NRN=spring beach. 
  - *Conditions:* 7 boats on the start. Poke went out on port. Rest on stbd. Corvo pinched us up. We tacked.  They stayed ahead. We did manage to pass insubordination on the downwind leg from R. 
- **2020-08-15**: Scatchet Head Tuneup with Erik
  - *Description:* Beautiful 80° day. Sailed from the hamburger to Skagit head and back took about six hours. We crossed all the way over to Apple Cove and cross back so that we could avoid some bad current near Skagit head
- **2020-08-15**: Scatchet Head Tuneup with Erik
  - *Description:* Beautiful 80° day. Sailed from the hamburger to Skagit head and back took about six hours. We crossed all the way over to Apple Cove and cross back so that we could avoid some bad current near Skagit head
- **2020-08-15**: Scatchet Head Tuneup with Erik
  - *Description:* Beautiful 80° day. Sailed from the hamburger to Skagit head and back took about six hours. We crossed all the way over to Apple Cove and cross back so that we could avoid some bad current near Skagit head
- **2020-08-17**: Ballard Cup Monday
  - *Description:* NRN.  12-18 knots.  Big sea state.  Main sail connection to the back of the boat broke as we warmed up and Paul fixed it. Then we won!  We got out well at start and then we pulled ahead at leward mark.  Stayed ahead  the whole way down.  
- **2020-08-17**: Ballard Cup Monday
  - *Description:* NRN.  12-18 knots.  Big sea state.  Main sail connection to the back of the boat broke as we warmed up and Paul fixed it. Then we won!  We got out well at start and then we pulled ahead at leward mark.  Stayed ahead  the whole way down.  
- **2020-08-21**: 2020-08-21
- **2020-08-22**: Styc singlehanded race
  - *Description:* Another beautiful day on the water. Course was NWRN. Paul did awesome!  5th place out of 27 boats. I was the race committee.  Alas the EIB I borrowed would not start after the race so I had to be towed in. I didn’t mind sitting. Andrea had time to chat. 
- **2020-08-26**: CYC Wednesday 
  - *Description:* Alex Samanis came with us. We did well up wind but corvo rounded before us at leward mark. Jube rounds third but beat us both!  Stayed at the rock wall side while we tacked west quick.   
  - *Conditions:* Rough water on stbd tack
- **2020-08-26**: CYC Wednesday 
  - *Description:* Alex Samanis came with us. We did well up wind but corvo rounded before us at leward mark. Jube rounds third but beat us both!  Stayed at the rock wall side while we tacked west quick.   
  - *Conditions:* Rough water on stbd tack
- **2020-08-26**: CYC Wednesday 
  - *Description:* Alex Samanis came with us. We did well up wind but corvo rounded before us at leward mark. Jube rounds third but beat us both!  Stayed at the rock wall side while we tacked west quick.   
  - *Conditions:* Rough water on stbd tack
- **2020-08-28**: Practice 
  - *Description:* We sailed with Antoine. 3 laps MEMEME
  - *Conditions:* 0-15 knots made for a challenge keeping up with shifting wind
- **2020-08-30**: Social sail
  - *Description:* Took the gps trackers out to test current.  Worked ok but harder to find than expected.  Need a flag or something unique standing up. 
  - *Conditions:* Shifting wind from many directions 
- **2020-08-30**: Social sail
  - *Description:* Took the gps trackers out to test current.  Worked ok but harder to find than expected.  Need a flag or something unique standing up. 
  - *Conditions:* Shifting wind from many directions 
- **2020-09-02**: 2020-09-02
- **2020-09-03**: Practice with April and Brian
  - *Description:* Did SBN course.  Fresh breeze at times. 
  - *Conditions:* Lovely night.  Moderate breeze. Some waves. 
- **2020-09-04**: Sunset/Moonlight Race
  - *Description:* Wow!  What a night!!!   Thanks so much to everyone for coming out and/or supporting this Moonlight Sail/Race.    I can't believe how nicely the wind cooperated!  Here is a recap.    Whisper, Gusto, Underdog, Poke, Creative and Peer got out there for some fun in the dark.   

The Sunset Race started promptly at 7:43pm going south NBMN rounding to stbd since the wind shift came in just in time.  Whisper and Gusto might have still been having fun rafted up at the start, but Peer and Underdog engaged in a serious race to the Ballard Sails buoy.  It was pretty hard to see by then and I mistook a kayak for a mark and called a tack too early.  Underdog took the mark first and we were playing catch up on the downwind leg.  Thanks to Jeane Goussev's awesome spin trimming, we crept up and passed Underdog somewhere around N, but when the wind shifted/reversed/played dead, we fouled the spinnaker and failed to save it in time so Underdog left us totally in the dust!  They were finishing as we rounded M.  So, huge Congrats to Lek, LA, and ?? (who your the third?) on Underdog for the win!  

The Moonlight Sail started promptly at 9:08pm at moonrise.  I tried to jump the gun since I was so excited to get going, but thx to wiser people we respected the real moonrise time!  The wind incredibly picked up to good moderate breeze 10-14knots after an early prediction of <4.  From left to right Poke, Creative, Peer and Underdog hit the Rock-N start line.    Creative rounded W just a bit out in front, and we found it hard to catch up with them but we tangled back and forth with Poke depending on who managed the 1.5knot current going northwest along the Magnolia bluffs better.   Port tack felt much better for sailing, but starboard was nice in the night since it really was hard to see the other boats at times.  As expected, the huge moon over downtown was a sight to see!  Such a clear beautiful night!  The ferries didn't really get in our way too much so it was easy sailing to D.    Now, which way were we rounding the D mark????  I won't say much here, but we rounded to strb with a bit of extra speed to make it around the big mark, had our spin set for a starboard raise and got into a good grove with 10-12 knots coming from about 173 degrees.  We were able to stay on that gybe all the way to W.  We crept up on Creative downwind, but darn they are hard to catch.  At some point we had lost track of both Underdog and Poke in the dark.  We were searching the sky and water for their little lights, but they might have been on the other side of a big ship parked in Elliot Bay.   Assuming it was just Creative we had to catch, we stayed focused on downwind speed running a bit hotter than polars directly to W.  Oh,  and let me point out that I did repeated tell my team that "this is a Sail not a Race", but they prompted told me that there were boats out there so they were racing.  There is just no way to shut off that instinct, so I finally put the popcorn away and got busier on main trim.  Well, out of the dark bluffs just shy of W, here comes Poke riding a 1.5+ knot current river along the coast.  They had sailed really deep on their symmetric spinnaker to take advantage of the current we all knew was there!  Smart sailing Elishia and Alex! They snuck in front of us, but didn't quite pass Creative and we ended up side by side back in the original left to right order Poke, Creative, Peer after the one and only gybe of the race all charging through the dark as the wind started picking up to a fresh breeze.   We stayed a bit high and were happy to have the right side since we like running that hot angle and we know some of that breeze was going to come around Magnolia Bluff.   Alas, the wind shift was still tough to handle, and I believe all 3 boats ended up with rudder out of water, on our ear, but under control!  Thanks, Amy Deckelbaum, for blowing the vang for us!!  Finally, there was the issue of finding the unlighted N buoy in the dark.  Harder than I expected!   Creative crossed the finish line first and I think we are going to have to check RaceQ's to see who gets the second place sticker that was promised.  Poke and Peer were right behind them!   Congrats to Creative!   

Congrats to all sailors out there for this epic summer adventure!  I really hope we can do it again sometime!  Send pics if you have some.  I'm attaching mine below.  I uploaded our track to RaceQ's and Underdog and Creative are there, but we are labeled as Unknown for now.  My track went past midnight so it's a bit screwed up.  Paul is working on fixing it!  Thx, Paul, you are an awesome sailing partner!  

Sailing fast on the warm southern breeze,
Sara
  - *Conditions:* Night time and twilght sailing.   Consistent breeze up wind, oscillating 168-195. Downwind we saw the usual massive wind shift that we see with strong breeze after rounding West Point and getting over toward the ship canal entrance.  It turns a southerly into an easterly so we are suddenly broaching.   So, stay high after rounding and leave yourself some room to go down as needed.  Maybe even aim high of the green can E, but no lower.
- **2020-09-07**: High Wind Practice
  - *Description:* Al, Shauna and LA came out with us for a practice with 20-35 knot winds predicted.  We saw up to 28 knots.  The course was over to Port Madison and back almost.  Wind angle was around 340.
  - *Conditions:* Big sea state, high winds, clear skies, minimal boat traffic.
- **2020-10-03**: Foul weather bluff
  - *Description:* Alas. Race canceled due to fog.  
  - *Conditions:* Soupy fog. No wind. 
- **2020-10-04**: Dry land practice 
  - *Description:* Talked through raises and windward and leward douse
  - *Conditions:* No wind or waves
- **2020-10-08**: Evening sail
  - *Description:* Left the dock around 6:30 PM as the sun was setting. We just did a reach out and back but the water was so quiet and we could hear little animal noises behind us like breathing it was just beautiful
  - *Conditions:* Dark sky.  Occasionally waves passed by. Wind 3-5
- **2020-10-10**: 2020-10-10
- **2020-10-11**: 2020-10-11
- **2020-10-17**: Styc fall regatta
  - *Description:* Three races. NERN, NBMN, NBWMN.  Wind was nice but shut off toward the end.  Luckily we made it around. Some boats came in an hour later. 
  - *Conditions:* 43 boats out there.  Nice wind. No rain but cloudy 
- **2020-10-24**: 2020-10-24
- **2020-10-25**: Pettit Prix Sunday 
  - *Description:* Beautiful day with good breeze.  The course was NUWN.  People to the right of us did better. Everyone zigzaged along the coast near Spring Beach a lot. U mark is black on top with yellow base.  
- **2020-11-01**: 2020-11-01
- **2020-11-02**: 2020-11-02
- **2020-11-03**: Monday social ssil
  - *Description:* Beautiful evening sail just after daylight savings time change.  Flat water. Nice wind.  No mistakes. 
  - *Conditions:* Gentle
- **2020-11-20**: Snowbird #1
  - *Description:* NMWN.  Started out light but soon picked up to 6 and then to 8.  High tide was at 10:30 and our start was 11:20. So current pushing north in middle but .5 knots south along rock wall. 
  - *Conditions:* Warm for November. 
- **2020-11-28**: In search of S
  - *Description:* Lovely day on the water. We went to find STYCs S mark off Skiff pt  on Bainbrige in Shallowater. Not the best spot for rounding a virtual mark but found something good nearby with a brown house on a rock wall just north of the point.   The house has stone steps that could line up with the north west corner of the house and a street coming down to a beach access. 
- **2020-11-29**: Number 98!
  - *Conditions:* Wind was 5-7. Nice sunset and full moon.  
- **2020-12-04**: Delivery to Tacoma
  - *Description:* Sailed down before the Vashon Island race from Tacoma.  Beautiful weather. More wind than expected so we filled up gas but didn’t need it.  And Whales!!!!  
  - *Conditions:* Down wind only for two days!
- **2020-12-05**: Winter Vashon Race
  - *Description:* Start in front of Tacoma Yatch Club. Go around Vashon clockwise. Race stopped early at northern end of Vashon so we headed home instead of spending another night in Tacoma. 
  - *Conditions:* Absolutely beautiful day!
- **2020-12-12**: 2020-12-12
- **2020-12-20**: 2020-12-20
- **2020-12-27**: 2020-12-27
- **2020-12-28**: 2020-12-28
- **2021-02-20**: Jim Dupue Race
  - *Description:* Port Madison yacht club race.  Used kwindo.  And they have a buoy over there!  Good to know!  Course was pt Madison to the red nun at Eagle Harbor to West Point to the new Z mark just west of Jefferson pt to start.  We did well to W but got trapped by a boat at rounding and just did not recover. Nice day back on the water. 
  - *Conditions:* Light air at first but wind picked up to max 12. Cloudy at times. No rain. No snow like last weekend. 
- **2021-02-27**: Practice 
  - *Description:* Windward leward practice with Amy on pit. She does well with clean up and processs.  Needs practice on timing.   I got to do main. 
  - *Conditions:* Nice day. Big puffy clouds. Winds from south.  Bit of a shadow near wall. 
- **2021-03-06**: 2021-03-06
- **2021-03-20**: SBYC Snowbird #5
  - *Description:* Crazy wind before and after wind but quieted down during race.  Almost didn’t make it by time limit.  NWMNx2
- **2021-03-26**: CYC Three Tree Point
  - *Description:* Beautiful day on water.  I messed everything up.  I didn’t do the spin switch right.  I chose wrong strategy.  I couldn’t handle Paul’s feedback.  

The boat felt good. At times we had good numbers.  Steve did really well on bow.  Gybes went smooth.  
- **2021-03-27**: 2021-03-27
- **2021-04-04**: 2021-04-04
- **2021-04-10**: 2021-04-10
- **2021-04-17**: PSSR Saturday 
  - *Description:* 4 races.  SAYF, SAYF, SAXAF, SAYF. Gorgeous day!  Wind was more than expected. 10-15 knots.  Minimal sea state considering it was ebbing. The starts were really tough. We were over early twice. Interesting since we often end up in second row.  
  - *Conditions:* Clear skies.  
- **2021-04-18**: PSSR Sunday 2021
  - *Description:* No races.  Wind was very light  but we tuned up with liftoff. 
- **2021-04-19**: Ballard Cup Monday
  - *Description:* NMBN. Light wind at start. Pin favored.  Nice port start above the whole fleet crossed to wall. Stayed right for better pressure and centered on the fleet.   Rounded nicely and quick gybe back to the wall.  Everyone caught us near B and in fact Puff came in last but with speed and rounded quick. Creative got tangled up with another boat. They got out of the stew better too. We did beat two j105s by better sailing on last leg.  
- **2021-04-21**: 2021-04-21
- **2021-04-23**: 2021-04-23
- **2021-04-24**: SYC Tri Island Series 1,to Double Bluff. 
  - *Description:* Windy rainy day and we double handed the 35 kn race.  Lousy start but caught up to everyone down wind.   Messy drop too close to Double Bluff. Tacked a lot with others to the point of exhaustion.  Fun day!
- **2021-04-26**: Styc Monday Ballard Cup
  - *Description:* Nice light wind day.  A lot of boats out there. Corvo still got us.  We might have beat Creative with Al on the boat but we might have been over early.  
- **2021-04-28**: 2021-04-28
- **2021-05-01**: 2021-05-01
- **2021-05-02**: 2021-05-02
- **2021-05-03**: STYC Monday
  - *Description:* Rain cleared just in time for the race.  Course was NBMNEMN
  - *Conditions:* Easterly wind and westerly current. 
- **2021-05-05**: 2021-05-05
- **2021-05-08**: 2021-05-08
- **2021-05-09**: 2021-05-09
- **2021-05-12**: CYC Wednesday
  - *Description:* Two races.  Rapidly changing conditions.  
  - *Conditions:* We saw over 15 before the first race.   Upped the shrouds.  And then it died to 10 or less for times.  Came up again at times.
- **2021-05-17**: 2021-05-17
- **2021-05-19**: 2021-05-19
- **2021-05-24**: 2021-05-24
- **2021-05-26**: 2021-05-26
- **2021-06-02**: 2021-06-02
- **2021-06-09**: 2021-06-09
- **2021-06-14**: 2021-06-14
- **2021-06-16**: 2021-06-16
- **2021-06-20**: 2021-06-20
- **2021-06-21**: RaceWeek Day 1
  - *Description:* Placing 7, 4, 7
  - *Conditions:* Sporty.  In the teens.
- **2021-06-22**: RaceWeek Day 2
  - *Description:* Placing 6, 5, 5
  - *Conditions:* Sporty.  In the teens.
- **2021-06-23**: RaceWeek Day 3
  - *Description:* Placing 8
  - *Conditions:* Very light.  
- **2021-06-24**: RaceWeek Day 4
  - *Description:* Placing 6 
  - *Conditions:* Light.  Tried two races.
- **2021-06-25**: RaceWeek Day 5
  - *Description:* Placing 7, 2
  - *Conditions:* Sporty.  In the teens. At times it looked like it would go lighter... but then it came roaring back.
- **2021-07-07**: 2021-07-07
- **2021-07-10**: 2021-07-10
- **2021-07-12**: 2021-07-12
- **2021-07-14**: 2021-07-14
- **2021-07-17**: 2021-07-17
- **2021-07-19**: 2021-07-19
- **2021-07-26**: 2021-07-26
- **2021-07-28**: 2021-07-28
- **2021-08-01**: 2021-08-01
- **2021-08-02**: 2021-08-02
- **2021-08-04**: 2021-08-04
- **2021-08-11**: 2021-08-11
- **2021-08-23**: 2021-08-23
- **2021-08-25**: 2021-08-25
- **2021-09-01**: 2021-09-01
- **2021-09-08**: 2021-09-08
- **2021-10-02**: Foul weather Bluff
  - *Description:* Light wind day predicted but we got 3-6 so enough for a race.  We had good position at leward mark but sailed into a hole and creative came from behind and passed us.  
  - *Conditions:* Sunny and nice gentle breeze 
- **2021-10-09**: 2021-10-09
- **2021-10-10**: PSSC Big Boat 
  - *Description:* Saturday to Sunday race with new North main.  Jack Christiansen came out Saturday when it was super windy but not on Sunday.  Sunday was nice. 4-7knots.  Mild sea state.  We got a 6th and a 2nd place.  The Buchans were racing Panic and won every race they did.  So our 2nd place was like a 1st in the fleet.  
  - *Conditions:* Busy.  Lots of boats.  Light wind. Some sprinkles. 
- **2021-10-14**: Fall regatta 
  - *Description:* 3 races.  NBWMN, NERN, NBMN. Mixed conditions.  5knots to 24 ish. At one point.  We got first in class and second over all.  Great day
  - *Conditions:* Mild sea state considering tide flooding and south wind
- **2021-10-16**: 2021-10-16
- **2021-10-22**: 2021-10-22
- **2021-10-23**: 2021-10-23
- **2021-10-24**: 2021-10-24
- **2021-10-31**: 2021-10-31
- **2022-03-05**: CSS Blakey Rock
  - *Description:* Start to N around Blakey Rocks to finish. Beautiful day.  Wind from North 335-38 more persistent than oscillation.  
  - *Conditions:* Mix of sun and storm clouds.  Sea state not too bad considering wind over 10 most of the time.  Lots of boats out so both rounding were a bit complex 
- **2022-03-19**: CSS #2
  - *Description:* Today was supposed to be to Scatchet Head and back, but wind was fluky so they set the course to be SKDF (start-Blakely Rocks-Duwamish Head ->finsish.
  - *Conditions:* fluky wind from n/s.  We saw 1knot and 18 knots. plenty in the 12-16 range.
- **2022-04-02**: 2022-04-02
- **2022-04-16**: 2022-04-16
- **2022-04-17**: 2022-04-17


,date,title,file,source
0,2019-10-04,Sail to Everett for Foulweather Bluff Race.,2019-10-04_16:43.pd.gz,loginfo
1,2019-10-05,Foulweather Bluff Race,2019-10-05_09:18.pd.gz,loginfo
2,2019-10-11,"Short practice, upwind tacks and downwind jibes.",2019-10-11_17:38.pd.gz,loginfo
3,2019-10-12,CYC PSSC Day 1,2019-10-12_09:45.pd.gz,loginfo
4,2019-10-18,"Short, at dock.",2019-10-18_13:51.pd.gz,loginfo
...,...,...,...,...
174,2022-03-05,CSS Blakey Rock,2022-03-05_09:13.pd.gz,gsheet
175,2022-03-19,CSS #2,2022-03-19_09:00.pd.gz,gsheet
176,2022-04-02,2022-04-02,2022-04-02_09:05.pd.gz,logprocess
177,2022-04-16,2022-04-16,2022-04-16_10:10.pd.gz,logprocess


In [9]:
# notebook - metadata schema... light.

# We currently do not have a schema for the metadata, it is implicit, which is dangerous.  We
# can extract fields and their types.
#
# Note, we make a simplifying assumption that we have three situations: 
# i) primitive types, ii) dicts, ii) list of dicts. 
#
# We do not have dicts containing dicts.

def schema_lite(race_data):
    "Find a lightweight schema from the existing data.  Provides a guide for future entries."
    fd = flatten_dicts("", race_data)
    return distill(fd)    

def distill(race_union):
    res = {}
    for k, v in race_union.items():
        if len(v) == 0:
            res[k] = v[0]
        elif isinstance(v[0], dict):
            # print(k, "collapsing")
            collapsed = collapse_dicts(v)
            # print(k, "distilling", collapsed)
            res[k] = distill(collapsed)
            # print(k, "done")
        else:
            res[k] = set([type(e) for e in v])
    return res

def flatten_dicts(prefix, dicts):
    "Pass over a list of dicts and extract fields, and collect all the values assined to those fields.  For fields which are list of dicts, recurse.  Combine and flatten all fields."
    res = {}
    for d in dicts:
        for k, v in d.items():
            if is_list_of_dicts(v):
                # print(k, v)
                v = flatten_dicts(k + "[]", v)
                res.update(v)
            else:
                res[prefix+k] = res.get(k, []) + [v]
    return res

def distill_types(race_union):
    "For each key in race_union return a set containing the "
    res = {}
    for k, v in race_union.items():
        res[k] = set([type(e) for e in v])
    return res

def add_key(res, key_list, val):
    if len(key_list) > 0:
        base_key = key_list[0]
        if len(key_list) == 1:
            res[base_key] = res.get(base_key, set()).union(val)
        else:
            next_dict = res.get(base_key, {})
            next_dict.update(add_key)
            if base_key in res:
                pass

### The fields in the metadata schema.

- The keys in this dict are the fields used.
   - If `foo[]bar` then field `foo` is a list of records, each containing the field `bar`.
- In both cases the value is a set of types that are encountered.


In [10]:
# notebook - extract the "schema"

schema_lite(metadata.records)

{'file': {str},
 'date': {str},
 'title': {str},
 'begin': {int},
 'end': {int},
 'source': {str},
 'description': {NoneType, str},
 'crew': {list},
 'purpose': {str},
 'conditions': {str},
 'performance': {str},
 'learnings': {str},
 'raceqs_video': {NoneType, str},
 'raceqs': {NoneType, str},
 'wind': {list},
 'shroud_name': {str},
 'shroud_tension': {list},
 'segments[]winds': {list},
 'segments[]tensions': {list},
 'segments[]port': {list},
 'segments[]stbd': {list},
 'segments[]thoughts': {str},
 'questions[]text': {str},
 'questions[]author': {str},
 'questions[]context': {str},
 'questions[]proposed_solution': {str},
 'observations': {str},
 'warnings': {str},
 'segments[]name': {str},
 'wave': {str},
 'settings': {str},
 'segments[]start': {datetime.datetime},
 'segments[]end': {datetime.datetime},
 'other': {str},
 'timestamp': {datetime.datetime},
 'shroud_tensions': {str},
 'port_pointing': {str},
 'stbd_pointing': {str},
 'fluids': {str},
 'fluid_comments': {str},
 'notes':

## Reading metadata from Google Forms

We created a Google form to speed up post race metadata entry: https://forms.gle/JENZZdSWKNuoF8icA

The advantage of this **public** form is that it is easy to enter data from any device (including mobile) at any time.  And the schema is at least "weakly" enforced.

The form provides a scheme for publishing the resulting data as a spreadsheet, which is here: https://docs.google.com/spreadsheets/d/e/2PACX-1vS5g8oeSAMk-CFP-xDi4hu9a23W-iF5SMNjap-Gd78BPWvhA1GGgpDqFkQaEUVD3zoM9Pud1fozuDn8/pub?output=csv

The steps are:
- Create a Google form which has the fields you want.
  - Get a Google account and drive.
  - Create the form here:  https://docs.google.com/forms/u/0/
- Form data can be viewed as a Google sheet
  - One column for each field.
- A Google sheet can be exported as a CSV file.

Note, the Google form is designed to be easy to use and enter data.  The ultimate goal is to produce semi-structured data that can be used to better analyze our performance in races.  The design of the form is a compromise that makes data entry easy (on the drive home from the race), and analysis easy.


In [11]:
# notebook - load GSHEET with form data

# This is a magic URL that can be extracted from any Google Sheet.  Look under File and then "Publish to Web".  
# Note this CSV takes a bit of time to be updated from the SHEET.
URL = r"https://docs.google.com/spreadsheets/d/e/2PACX-1vS5g8oeSAMk-CFP-xDi4hu9a23W-iF5SMNjap-Gd78BPWvhA1GGgpDqFkQaEUVD3zoM9Pud1fozuDn8/pub?output=csv"

# Use pandas to read the CSV
raw_gsheet = pd.read_csv(URL)  # index_col=0, parse_dates=True)

display_markdown("### A list of the columns in the GSHEET")

for c in raw_gsheet.columns:
    print(repr(c))


### A list of the columns in the GSHEET

'Timestamp'
'Date YYYY-MM-DD (e.g. "2020-05-10") '
'Title: short name for sail (e.g. SBYC Snowbird #1)'
'Description (2-3 sentences, optional)'
'Purpose'
'Winds in knots (pick best description)'
'Wave height (pick best)'
'Conditions (i.e. description)'
'Settings Summary (how were sail controls set? trim?)'
'Shrouds (short name)'
'Shroud tensions (UP-MID-LOW: sep with hyphens: 29-10-0). Pos low is tension, neg low is circle size in cm).'
'Crew'
'Additional Crew (comma separated)'
'Performance (i.e. how did we perform vs other boats or polars).'
"Learnings (something we'd like to repeat or avoid)"
'Other (try to be structured!)'
'Port Pointing'
'Starboard Pointing'
'Gas, Water, Pump Out, Empty Bilge?'
'Comments on Fluids?'
'Warnings (needed repair, change, etc).'


In [12]:
# The column names are long and verbose, because the SHEET column names are the form field prompts.

# These names are good as documentation, but painful for programmatic access.  The table below maps from 
# a long name and a convenient short form.

SHORT_COLNAME_TO_LONG = {
    'date'            : 'Date YYYY-MM-DD (e.g. "2020-05-10") ',
    'title'           : 'Title: short name for sail (e.g. SBYC Snowbird #1)', 
    'purpose'         : 'Purpose',
    'crew'            : 'Crew',
    'description'     : 'Description (2-3 sentences, optional)',
    'conditions'      : 'Conditions (i.e. description)',
    'performance'     : 'Performance (i.e. how did we perform vs other boats or polars).',
    'learnings'       : "Learnings (something we'd like to repeat or avoid)",
    'warnings'        : 'Warnings (needed repair, change, etc).',
    'wave'            : 'Wave height (pick best)',
    'wind'            : 'Winds in knots (pick best description)',
    'port_pointing'   : 'Port Pointing',
    'stbd_pointing'   : 'Starboard Pointing',
    'settings'        : 'Settings Summary (how were sail controls set? trim?)',
    'shroud_name'     : 'Shrouds (short name)',
    'shroud_tension'  : 'Shroud tensions (UP, MID, LOW: comma sep: 29,10,0). Pos low is tension, neg low is circle size in cm).',
    'other'           : 'Other (try to be structured!)',
    'additional_crew' : 'Additional Crew (comma separated)',
    'timestamp'       : 'Timestamp',
    'fluid_comments'  : 'Comments on Fluids?',
    'fluids'          : 'Gas, Water, Pump Out, Empty Bilge?',
}

LONG_COLNAME_TO_SHORT = {v:k for k, v in SHORT_COLNAME_TO_LONG.items()}

In [13]:
# Read the sheet and convert to a pandas table.

def read_gsheet():
    "Read the latest GSHEET.  Check that nothing bad has happened, and convert to short names."
    gs = pd.read_csv(G.GSHEET_URL)
    check_columns_changed(gs, LONG_COLNAME_TO_SHORT)
    return convert_to_short_names(gs)

def check_columns_changed(df, long_colname_map):
    """
    Its entirely possible that I will someday edit the form and then the columns will get
    out of whack.  Check to see that there are neither new fields or missing fields.
    """
    new_colnames = []
    missing_colnames = []
    for c in df.columns:
        if c not in long_colname_map:
            new_colnames.append(c)
    for c in long_colname_map:
        if c not in df.columns:
            missing_colnames.append(c)
    if len(new_colnames) == 0 and len(missing_colnames) == 0:
        G.logger.info("No missing or extra columns.")
        return True
    else:
        G.logger.warning(f"Uh Oh. New cols {new_colnames}. Missing cols {missing_colnames}.")
        return False

def convert_to_short_names(raw_metadata):
    "Assuming the columns names have not changed, convert to a short form."
    return raw_metadata.rename(LONG_COLNAME_TO_SHORT, axis='columns')

In [14]:
# notebook - read the sheet and display the rows

gsheet = read_gsheet()
gsheet

2022-04-21 07:54:31,600|WARNING|check_columns_changed| Uh Oh. New cols ['Shroud tensions (UP-MID-LOW: sep with hyphens: 29-10-0). Pos low is tension, neg low is circle size in cm).']. Missing cols ['Shroud tensions (UP, MID, LOW: comma sep: 29,10,0). Pos low is tension, neg low is circle size in cm).'].


,timestamp,date,title,description,purpose,wind,wave,conditions,settings,shroud_name,...,crew,additional_crew,performance,learnings,other,port_pointing,stbd_pointing,fluids,fluid_comments,warnings
0,5/8/2020 9:37:40,2020-05-06,Practice,Went out with Marisa to practice for cohabitat...,practice,8-14,1-2,Great,Not quite right,Light wind,...,"Sara, Marisa, Paul",NaN,Not measurd,Departure was tricky with strong wind from nor...,NaN,NaN,NaN,NaN,NaN,NaN
1,5/8/2020 14:50:22,2020-04-16,Tune-up with Creative,"Upwind to Pt. Wells buoy, raised and raced hom...",tune_up,4-10,flat,Beautiful day. Winds were 3 quickly building t...,NaN,"New ""base""?",...,"Sara, Paul",NaN,Good height and speed vs. Creative on the way ...,Let the sails out for downwind: both main and ...,NaN,NaN,NaN,NaN,NaN,NaN
2,5/8/2020 21:32:07,2020-05-08,Practice,Good day at about 13 knots. Nice weather. J...,practice,8-14,1-2,Weird wind shift at the end of our excursion,3 inches on the back stay main ion on tight Cu...,Base,...,"Sara, Paul",NaN,No other boats out there.,We can do a leward drop.,Prep for Cohabitation Race after lockdown. Lo...,NaN,NaN,NaN,NaN,NaN
3,5/9/2020 16:37:05,2020-05-09,Cohabitate Race,Great day on the water. Range from 10-15 Jordi...,race,8-14,1-2,Oscillating wind from the north.,It was trimmed out well. All adjusted the uppe...,Modified,...,"Sara, Marisa, Paul",NaN,Came in second place behind Poke and Dystroy. ...,Learn the letterbox drop. Don’t Try new thing...,Lube turnbuckles,NaN,NaN,NaN,NaN,NaN
4,5/16/2020 23:37:18,2020-05-16,Cohab 2 race,Paul organized this race. The course was nwmn ...,race,0-6,flat,Very light to no wind at times. Strong current...,Lots of twist. No back stay. No Cunningham,Extra light wind. No yet on chart.,...,"Sara, Paul",NaN,Good on down wind. Recovered from a bad start ...,If you can’t take because the wind is so light...,Engine ran well,28-30,28-30,Empty Bilge?,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,4/17/2022 22:28:26,2022-04-17,CYC Spring Regatta 2022,Two days of racing this weekend. Fresh breeze ...,race,4-10,flat,Clear skies wind starting from 180 in south ar...,Light. Main halyard at 6. Jib a bit loose. C...,Light air,...,"Sara, Paul, Peter","Peter Colleran, Peter Rostas, Ruby",Slowish all weekend. Needed bottom cleaning.,Discuss plan if protest called on us. Yell pr...,NaN,28-30,28-30,Empty Bilge?,Checked oil. One centimeter below full,Jib furler was not working well. Bringing jib...
81,7/4/2021 13:15:30,2021-06-22,RaceWeek Day 2,"Placing 6, 5, 5",race,12-18,2-4,Sporty. In the teens.,Base,Base,...,"Sara, Paul, Amy, Steve",NaN,Generally well upwind and down.,Need to focus on sailing high mode.,NaN,25-27,25-27,NaN,NaN,NaN
82,7/4/2021 13:15:40,2021-06-23,RaceWeek Day 3,Placing 8,race,0-6,1-2,Very light.,Light Air,Base,...,"Sara, Paul, Amy, Steve",NaN,Generally well upwind and down.,NaN,NaN,25-27,25-27,NaN,NaN,NaN
83,7/4/2021 13:15:45,2021-06-24,RaceWeek Day 4,Placing 6,race,4-10,1-2,Light. Tried two races.,Light Wind,Base,...,"Sara, Paul, Amy, Steve",NaN,Generally well upwind and down.,NaN,NaN,25-27,25-27,NaN,NaN,NaN


In [15]:
# Next step is to convert the Google Form spreadsheet rows to race metadata entries.  The goal was to 
# keep the two "close" so that conversion is not onerous,  but we do need to massage some of the fields.

def gsheet_row_to_metadata(row):
    "Convert the Google Form spreadsheet rows to race metadata entries."
    res = {}
    # If both the date and the timestamp is missing then something must be wrong.
    if is_missing_value(row['date']) and is_missing_value(row['timestamp']):
        G.logger.info(f"Encountered a row with missing date and timestamp. Skipping.")
        return None
    for key, val in row.iteritems():
        if key == 'date' and is_missing_value(val):
            val = timestamp_convert(row['timestamp']).format("YYYY-MM-DD")
        if is_missing_value(val):
            continue
        elif key == 'timestamp':
            val = timestamp_convert(row['timestamp']).datetime
        elif key == 'wind':
            val = [int(s.strip()) for s in val.split("-")]
        elif key == 'crew':
            val = [s.strip() for s in val.split(",")]
        elif key == 'shroud_tension':
            val = [int(s.strip()) for s in val.split(",")]
        res[key] = val
    res['source'] = 'gsheet'
    # It is a bit easier if the fields are in the same/similar order.
    return reorder_some_keys(res, SHORT_COLNAME_TO_LONG.keys())

def reorder_some_keys(dictionary, keys):
    "Return a dictionary with the keys in the order presented in keys."
    d = copy.copy(dictionary)
    res = dict()
    # Copy the ones in keys
    for k in keys:
        if k in d:
            res[k] = d.pop(k)
    # Copy the rest.
    for k in d.keys():
        res[k] = d[k]
    return res

def is_missing_value(val):
    "Pandas replaces empty CSV entries with NaN.  Return True if encountered."
    return isinstance(val, numbers.Number) and np.isnan(val)

def timestamp_convert(val):
    "Convert the google forms timestamp to a date."
    return arrow.get(val, 'M/D/YYYY H:mm:ss', tzinfo='US/Pacific')

In [16]:
# notebook - convert from gsheet to metadata format

gsheet_records = [gsheet_row_to_metadata(row) for index, row in gsheet.iterrows()]

# Show one row.
print(save_yaml(gsheet_records[-1:]))

date: '2021-06-25'
title: RaceWeek Day 5
purpose: race
crew:
- Sara
- Paul
- Amy
- Steve
description: Placing 7, 2
conditions: Sporty.  In the teens. At times it looked like it would go lighter...
  but then it came roaring back.
performance: 'Generally well upwind and down. '
learnings: Nice job in trick conditions.
wave: 2-4
wind:
- 12
- 18
port_pointing: 25-27
stbd_pointing: 25-27
settings: Base
shroud_name: Base
timestamp: 2021-07-04 13:15:50-07:00
source: gsheet



In [17]:
# If there are new rows in the GSHEET, then add them to metadata.

def update_metadata_from_gsheet():
    "Read metadata.yml and gsheet and update as needed."
    metadata = read_metadata()
    gsheet = read_gsheet()
    new_records = add_gsheet_records(gsheet, metadata)
    save_metadata(new_records)

def add_gsheet_records(gsheet, metadata):
    "Find records in the gsheet which are missing from the existing metadata."
    dates = metadata.dates.copy()
    res = []
    for _, row in gsheet.iterrows():
        record = gsheet_row_to_metadata(row)
        if record is not None:
            date = record['date']
            G.logger.debug(f"Examining record {date}")
            if date not in dates:
                # If the date is missing just add it.
                G.logger.info(f"Found new record for {date} : {record.get('title', '')}")
                res.append(record)
            else:
                # If the date is already present we need to merge.
                existing = dates.pop(date)
                source = existing['source']
                if source == 'byhand':
                    # The existing was entered byhand in metadata.yml.  Be careful!
                    if metadata.timestamp < record['timestamp']:
                        G.logger.warning(f"Duplicate record. GSheet row is newer than byhand metadata: {record['timestamp']}.")
                        # Append both, we'll need to figure this out by hand
                        res.append(existing)
                        res.append(record)
                    else:
                        # Otherwise the GSHEET entry is older than the current record.  Ignore.
                        res.append(existing)
                elif source in ['loginfo', 'logprocess', 'gsheet']:
                    # Source was an automated process.  We can merge the records, overwriting with GSHEET
                    G.logger.debug(f"Merging gsheet into exiting record.")
                    # Overwrite the values in these records.
                    new_record = {**existing, **record}
                    res.append(new_record)
                else:
                    G.logger.warning(f"Found strange source: {source}.")
    return res + list(dates.values())

# notebook

In [18]:
#notebook 

import collections

def find_duplicates(metadata):
    "Find duplicate dates in the metadata."
    dates = collections.defaultdict(list)
    for record in metadata.records:
        date = record['date']
        dates[date] += [record]
    return {k:v for k, v in dates.items() if len(v) > 1}
        

def find_missing_data(metadata):
    records = []
    for record in metadata.records:
        file = record.get('file', None)
        if file is None:
            records.append(record)
    return records
        
    
metadata = read_metadata()

dups = find_duplicates(metadata)
print(dups.keys())

print([(r['date'], r['title']) for r in find_missing_data(metadata)])

2022-04-21 07:54:35,164|INFO|read_metadata| Read 179 records.


dict_keys(['2020-08-08', '2020-08-15', '2020-08-17', '2020-08-26', '2020-08-30'])
[('2020-06-01', 'Monday race'), ('2020-06-24', 'Gas run'), ('2020-06-29', 'TYT Monday night '), ('2020-07-27', 'Ballard cup Monday '), ('2020-07-31', 'Picnic Sail Friday'), ('2020-08-08', 'Double practice day!!'), ('2020-08-15', 'Scatchet Head Tuneup with Erik'), ('2020-08-15', 'Scatchet Head Tuneup with Erik'), ('2020-08-17', 'Ballard Cup Monday'), ('2020-08-26', 'CYC Wednesday '), ('2020-08-26', 'CYC Wednesday '), ('2020-08-30', 'Social sail'), ('2020-09-04', 'Sunset/Moonlight Race'), ('2020-10-04', 'Dry land practice '), ('2020-10-08', 'Evening sail'), ('2020-11-03', 'Monday social ssil'), ('2020-11-20', 'Snowbird #1'), ('2020-12-05', 'Winter Vashon Race'), ('2021-03-26', 'CYC Three Tree Point'), ('2021-04-24', 'SYC Tri Island Series 1,to Double Bluff. '), ('2021-05-12', 'CYC Wednesday'), ('2021-06-21', 'RaceWeek Day 1'), ('2021-06-24', 'RaceWeek Day 4'), ('2021-06-25', 'RaceWeek Day 5'), ('2021-10-14'

In [19]:
# notebook 

# loginfo is a legacy location for metadata, need to pull that in...  once.

log_info = pd.read_pickle(G.LOG_INFO_PATH)
display(len(log_info))
log_info[:5]


43

,file,race,begin,end,datetime,description
0,2020-05-01_16:46.pd.gz,,0,-1,2020-05-01 16:46:00-07:00,
1,2020-04-28_16:43.pd.gz,,0,-1,2020-04-28 16:43:00-07:00,
2,2020-04-26_14:26.pd.gz,,0,-1,2020-04-26 14:26:00-07:00,
3,2020-04-24_15:10.pd.gz,,32140,130560,2020-04-24 15:10:00-07:00,Tune Up with Creative
4,2020-04-19_12:00.pd.gz,,0,-1,2020-04-19 12:00:00-07:00,Tune Up with Creative


In [20]:
def update_metadata_from_loginfo():
    "Read metadata.yml and loginfo and update as needed."
    metadata = read_metadata()
    loginfo = pd.read_pickle(G.LOG_INFO_PATH)    
    updated = merge_loginfo_records(loginfo, metadata)
    save_metadata(updated)


# Process the legacy loginfo data, this is only needed once.
def merge_loginfo_records(loginfo, metadata):
    """
    Create a metadata record for each loginfo record.  When a key already exists in
    metadata merge the info, overwriting fields in the loginfo record.
    """
    dates = metadata.dates.copy()
    rows = []
    for i, row in loginfo.iterrows():
        adt = datetime_from_log_filename(row.file)
        date_string = date_from_datetime(adt)
        record = {}
        # Munge loginfo data into "metadata" schema.
        record['file'] = row.file
        record['date'] = date_string
        record['title'], record['description'] = loginfo_title(row)
        record['begin'] = row.begin
        record['end'] = row.end
        record['source'] = 'loginfo'
        # Overwrite with the existing record... if it exists.
        if date_string in dates:
            record.update(dates.pop(date_string))
        rows.append(record)
    return rows + list(dates.values())

def loginfo_title(row):
    "Create a title and description from row record."
    if len(row.race) > 0:
        return row.race, row.description
    else:
        return row.description, ''


In [21]:
# Finally, during upload we should ensure that there is a default and empty metadata record 
# for each race.

def add_missing_metadata():
    """
    Working backward from the full list of pandas datafiles, ensure there is a default
    entry in the metadata file for each.
    """
    metadata = read_metadata()
    dates = metadata.dates.copy()
    new_dates = {}
    pfiles = p.pandas_files()
    G.logger.info(f"Found {len(pfiles)} pandas files.")
    for f in sorted(pfiles):
        adt = datetime_from_log_filename(f)
        date = date_from_datetime(adt)
        G.logger.debug(f"Examining {date} : {f}")
        # Default metadata... basically empty
        record = dict(file=f, date=date, title=date, begin=0, end=-1, source='logprocess')
        if date not in dates:
            G.logger.info(f"Found missing entry for {date} : {f}.")
        if date in new_dates:
            G.logger.warning(f"Two files for {date}. Watch out. Skipping.")
        else:
            existing = dates.pop(date, {})
            record.update(existing)
            new_dates[date] = record
    all_records = list(new_dates.values()) + list(dates.values())
    G.logger.info(f"Outputting {len(all_records)} records.")
    save_metadata(all_records)


def datetime_from_log_filename(filename, time_zone='US/Pacific'):
    "Extracts the datetime from a log filename."
    dt_string = re.sub(r".gz$", "", filename)   # Could be compressed
    dt_string = re.sub(r".pd$", "", dt_string)  # Standard .pd
    return arrow.get(dt_string, "YYYY-MM-DD_HH:mm", tzinfo=time_zone)
    

def date_from_datetime(adt):
    return adt.format("YYYY-MM-DD")

In [22]:
def update_race(updated_race_record):
    "Replace the race record, by date."
    date = updated_race_record['date']
    md = read_metadata()
    if date not in md.dates:
        raise Exception(f"Warning, {date} could not be found in the race logs.")
    md.dates[date] = updated_race_record
    save_metadata(list(md.dates.values()))

In [23]:
# notebook - 

if False:
    update_metadata_from_loginfo()
    add_missing_metadata()
    update_metadata_from_gsheet()